In [93]:
'''
Extrinsic Matrices
'''
import numpy as np

# Robot Height
robot_height = 10.9

# Outpost 1 Camera position
o1_point = (-8.98976, 177.5661, 457.6042)

# Outpost 1 Camera to World Matrix
# -0.70486 0.22467    -0.67282   -8.98975
# -0.02764 0.93910    0.34254    177.56610
# -0.70880 -0.26004   0.65573    457.60420
# 0.00000  0.00000    0.00000    1.00000
c2w_matrix_1 = np.array([[-0.70486, 0.22467, -0.67282, -8.98975],
                         [-0.02764, 0.93910, 0.34254, 177.56610],
                         [-0.70880, -0.26004, 0.65573, 457.60420],
                         [0.00000, 0.00000, 0.00000, 1.00000]])

# Outpost 1 Camera Projection Matrix
# 0.87706	0.00000	0.00000	0.00000
# 0.00000	0.49335	0.00000	0.00000
# 0.00000	0.00000	0.00000	-1.00000
# 0.00000	0.00000	-1.66617	1.66717
projection_inv_1 = np.array([[0.87706,	0.00000, 0.00000, 0.00000],
                                [0.00000,	0.49335, 0.00000, 0.00000],
                                [0.00000,	0.00000, 0.00000, -1.00000],
                                [0.00000,	0.00000, -1.66617, 1.66717]])

# Outpost 3 Camera position
o3_point = (817.1082, 177.5661, -9.41583)

# Outpost 3 Camera to World Matrix
# 0.69863  -0.23571   0.67554    817.10820
# -0.03390 0.93221    0.36032    177.56610
# 0.71468  0.27463    -0.64328   -9.41580
# 0.00000  0.00000    0.00000    1.00000
c2w_matrix_3 = np.array([[0.69863, -0.23571, 0.67554, 817.10820],
                         [-0.03390, 0.93221, 0.36032, 177.56610],
                         [0.71468, 0.27463, -0.64328, -9.41580],
                         [0.00000, 0.00000, 0.00000, 1.00000]])

# Outpost 3 Camera Projection Matrix Inverse
# 0.87706	0.00000	0.00000	0.00000
# 0.00000	0.49335	0.00000	0.00000
# 0.00000	0.00000	0.00000	-1.00000
# 0.00000	0.00000	-1.66617	1.66717
projection_inv_3 = np.array([[0.87706,	0.00000, 0.00000, 0.00000],
                                [0.00000,	0.49335, 0.00000, 0.00000],
                                [0.00000,	0.00000, 0.00000, -1.00000],
                                [0.00000,	0.00000, -1.66617, 1.66717]])

matrix_1 = np.dot(c2w_matrix_1, projection_inv_1)
matrix_3 = np.dot(c2w_matrix_3, projection_inv_3)

In [98]:
'''
convert screen point to robot point for Outpost 3 - Screen 2
'''
def s2r_o3(screen_point, s2w_matrix, camera_point):
    (sx, sy) = screen_point
    plane_point = np.array([(sx*2-1)*1000, ((1-sy)*2-1)*1000, 1000, 1000])
    world_point = np.dot(s2w_matrix, plane_point)
    world_point = world_point[:-1].tolist()
    print(world_point)

    (x1, y1, z1) = camera_point
    (x2, y2, z2) = world_point
    a1 = y1 - robot_height
    a2 = y1 - y2
    xr = x1 + a1/a2 * (x2-x1)
    zr = z1 + a1/a2 * (z2-z1)
    return xr, zr

# mid_point1 = s2r_o3((0.5,0.5), matrix_1, o1_point)
# mid_point3 = s2r_o3((0.5,0.5), matrix_3, o3_point)
#
# print("mid point 1", mid_point1, sep='\t')
# print("mid_point3", mid_point3, sep='\t')

test_point_0 = s2r_o3((0.33828125000000003, 0.7375), matrix_3, o3_point)
print("test_point_0", test_point_0, sep='\t')

[-1.3784560789354146, -391.59260488435393, 366.77086119125306]
test_point_0	(577.4316779683548, 100.74249342490464)


(578.7988, 99.15597)

In [92]:
'''
说明了 wp -> sp 可行, 且误差极小 => 但是可能需要考虑如何平衡两个 Camera 的参数
'''
w2c_matrix_3 = np.linalg.inv(c2w_matrix_3)
projection_3 = np.linalg.inv(projection_inv_3)
matrix = np.dot(projection_3, w2c_matrix_3)

def test_1(wp):
    sp = np.dot(matrix, wp)
    # print("output sp", sp, sep='\t')
    # k = sp[2]
    sp = sp/sp[2]
    # print("x", sp[0], "y", sp[1], sp[2], sp[3], sep='\t')
    # print("wp", np.dot(np.linalg.inv(matrix), sp), sep='\t')
    # print("processed sp", (sp[0]+1)/2, 1 - (sp[1] + 1)/2, sep='\t')
    print((sp[0] + 1)/2 * 1280, 720 - (sp[1] + 1)/2 * 720, sep='\t')

def test(x):
    [x1, x2] = x
    print("input sp", [x1 * 1280 ,x2 * 720], '\n', sep='\t')

# b1
test_1(np.array([578.7988, 10.9, 99.15597, 1]))
test([0.33828125000000003, 0.7375])
# b2
test_1(np.array([555.8079, 10.9, 341.1539, 1]))
test([0.589453125, 0.4826388888888889])
# r1
test_1(np.array([289.7958,10.9, 56.153, 1]))
test([0.10859375, 0.5201388888888889])
# r2
test_1(np.array([189.1591, 10.9, 334.1981, 1]))
test([0.35000000000000003, 0.36388888888888893])

430.86207776967217	534.2992993425983
input sp	[433.00000000000006, 531.0]	

756.3736363995108	356.049756898566
input sp	[754.5, 347.5]	

136.85691677183945	380.8172565603058
input sp	[139.0, 374.5]	

445.9823021427974	269.95560424260947
input sp	[448.00000000000006, 262.0]	



In [80]:
def sp2wp(sp):
    x = 2*sp[0] - 1
    y = 2*(1-sp[1]) - 1
    print(x, y, sep='\t')
    wp = np.dot(np.linalg.inv(matrix), np.array([x,y,1.0,1.0014652526237025]))
    wp = wp * 10.9 / wp[1]

    print(wp)

sp2wp([0.33828125000000003, 0.7375])

-0.32343749999999993	-0.4750000000000001
[521.85194479  10.9         90.22931975   0.90116831]
